In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, ZeroPadding2D, InputLayer, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.applications.mobilenet as mn


import skimage.io
from selective_search import selective_search, box_filter

import os
from os import path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import cv2
import json
import random
import tqdm

In [2]:
from nms import nms

In [232]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(40, 40, 3)))
model.add(Conv2D(64, 3,activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [171]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', 
           input_shape=(40, 40 ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(units=2, activation="softmax")
])


In [8]:
PATH = "/mnt/d/personal/cotton"

In [240]:
model.load_weights("final_model-14.hdf5")

In [124]:
model.load_weights("mn-50.hdf5")

In [125]:
# Runs the model through the sample images and draws a bounding box
do_nms = True
do_prediction = False
prob_thresh = 0.9999
stride = 30
shape = 60
base_path = path.join(PATH, "samples")
ann = pd.read_csv(path.join(base_path, "via_export_csv.csv"))
for file in ann["filename"].unique():
    cnt = 0
#     if file not in ["2019_1010_123336_234_7_4.jpg", "2019_1010_123350_243_5_3.jpg", "2019_1010_123336_234_9_8.jpg", "2019_1010_123351_244_4_5.jpg"]:
#         continue
    if not file.startswith("2019_1010_123350_243"):
        continue
#     if not file.startswith("2019_1010_123351_244"):
#         continue
#     if not file.startswith("2019_1010_123336_234"):
#         continue
    sample_ann = ann[ann["filename"] == file]
    training_data = []
    img = cv2.imread(path.join(base_path, file))
    for _x in range(0, img.shape[1] - shape, stride):
        for _y in range(0, img.shape[0] - shape, stride):
            xstart = _x
            xend = _x + shape
            ystart = _y
            yend = _y + shape
            training_data.append([ystart, yend, xstart, xend])
    
    img = cv2.imread(path.join(base_path, file))
    if do_prediction:
        all_images = []
        predictions = []
        for ix in tqdm.tqdm_notebook(range(len(training_data))):
            val = training_data[ix]
            samp_img = img[val[0]: val[1], val[2]: val[3]]
            if samp_img.shape[0] != 60 or samp_img.shape[1] != 60:
                all_images.append(cv2.resize(samp_img, (60, 60), interpolation=cv2.INTER_AREA))
            else:
                all_images.append(samp_img)
            if len(all_images) == 50000:
                all_images = np.array(all_images) * 1.0 / 255
                temp_predictions = model.predict(all_images, verbose=1).tolist()
                predictions.extend(temp_predictions)
                all_images = []
        all_images = np.array(all_images) * 1.0 / 255
        predictions.extend(model.predict(all_images, verbose =1).tolist())
    gt = 0
    for ix, val in sample_ann.iterrows():
        coords = json.loads(val["region_shape_attributes"])
        if "x" not in coords.keys():
            continue
        x1 = coords["x"]
        if x1 is None:
            continue
        x2 = x1 + coords["width"]
        y1 = coords["y"]
        y2 = y1 + coords["height"]
        img_crop = img[y1:y2, x1:x2]
        img[y1:y2, x1:x2] = np.ones_like(img_crop)
        gt += 1
        
    imgg = cv2.imread(path.join(base_path, "labels", file))
    all_boxes = []
    all_scores = []
    for ix in range(len(predictions)):
        prob = predictions[ix][1]
        if prob > 0.5 or prob > prob_thresh:
            val = training_data[ix]
            all_boxes.append([val[2], val[0], shape, shape])
            all_scores.append(prob)
    if do_nms:
        indices = nms.boxes(all_boxes, all_scores, nms_threshold=0.1)
    else:
        indices = range(len(all_boxes))

    cp = 0
    wp = 0
    for ix in indices:
        if all_scores[ix] > prob_thresh:
            cnt += 1
            x1 = all_boxes[ix][0]
            y1 = all_boxes[ix][1]
            x2 = x1 + shape
            y2 = y1 + shape
            img_crop = img[y1:y2, x1:x2]
            if len(np.where(img_crop==1)[0]) > 500:
                color = (0, 255, 0)
                cp += 1
            else:
                color = (255, 0, 255)
                wp += 1
            imgg = cv2.rectangle(imgg, (x1, y1),
                                 (x2, y2),
                                 color, 2)
    cv2.imwrite(path.join(base_path, "predictions", file), imgg)
    print("GT: ", gt)
    print("Predictions: ", cnt, cp/cnt*100)
    print("Correct Predictions: ", cp, cp/gt*100)
    print("Wrong Prediction: ", wp)

GT:  59
Predictions:  82 39.02439024390244
Correct Predictions:  32 54.23728813559322
Wrong Prediction:  50


In [255]:
rez_img = cv2.resize(imgg, (2000, 1500))

In [256]:
cv2.imwrite(path.join(base_path, "predictions", "2019_1010_123351_244_cmp_vgg14.jpg"), rez_img)

True

In [149]:
mdl = mn.MobileNet(weights=None)

In [119]:
fin = mdl.layers[-1]

In [140]:
inp = InputLayer(input_shape=(40, 40, 3))

In [156]:
inp = Input(shape=(40, 40, 3))

In [90]:
model = mn.MobileNet(input_shape=(60, 60, 3), classes=2, weights=None)

In [170]:
new_mdl.summary()

Model: "mobilenet_1.00_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 40, 40, 3)]       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 41, 41, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 20, 20, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 20, 20, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 20, 20, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 20, 20, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 20, 20, 32)  

In [129]:
mdl.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [116]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_10 (ZeroPaddi (None, 42, 42, 3)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 40, 40, 64)        1792      
_________________________________________________________________
zero_padding2d_11 (ZeroPaddi (None, 42, 42, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 40, 40, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
zero_padding2d_12 (ZeroPaddi (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 20, 20, 128)      

In [21]:
old_prob

0.99

#### Ep 14:
GT:  59
Predictions:  121 38.84297520661157
Correct Predictions:  47 79.66101694915254
Wrong Prediction:  74

#### Ep 6:
GT:  59
Predictions:  88 43.18181818181818
Correct Predictions:  38 64.40677966101694
Wrong Prediction:  50

#### Final 2
GT:  59
Predictions:  103 34.95145631067961
Correct Predictions:  36 61.016949152542374
Wrong Prediction:  67

#### Final 1
GT:  59
Predictions:  93 38.70967741935484
Correct Predictions:  36 61.016949152542374
Wrong Prediction:  57

#### Final 3
GT:  59
Predictions:  83 38.55421686746988
Correct Predictions:  32 54.23728813559322
Wrong Prediction:  51

#### Final 4
GT:  59
Predictions:  139 28.05755395683453
Correct Predictions:  39 66.10169491525424
Wrong Prediction:  100

#### model vgg 20
GT:  59
Predictions:  250 23.599999999999998
Correct Predictions:  59 100.0
Wrong Prediction:  191
GT:  96
Predictions:  243 31.68724279835391
Correct Predictions:  77 80.20833333333334
Wrong Prediction:  166